In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
train_paht='../input/chest-ctscan-images/Data/train'
test_path='../input/chest-ctscan-images/Data/test'
validation_path='../input/chest-ctscan-images/Data/valid'

In [18]:
train_datagen=ImageDataGenerator(rescale=1/255, vertical_flip=True, fill_mode='reflect')
test_datagen=ImageDataGenerator(rescale=1/255,  vertical_flip=True, fill_mode='reflect')
validation_datagen=ImageDataGenerator(rescale=1/255, vertical_flip=True, fill_mode='reflect')

In [19]:
train_geneator=train_datagen.flow_from_directory(train_paht,
                                                target_size=(200,200),
                                                color_mode='rgb',
                                                 batch_size=128,
                                                subset='training',
                                                class_mode='categorical')
test_generator=test_datagen.flow_from_directory(test_path,
                                               target_size=(200,200),
                                               color_mode='rgb',
                                                batch_size=128,
                                               class_mode='categorical')
                                             
validation_generator=validation_datagen.flow_from_directory(validation_path,
                                                           target_size=(200,200),
                                                            color_mode='rgb',
                                                            batch_size=128,
                                                            class_mode='categorical')

In [10]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
checkpoint=ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True)

In [39]:
pre_trained_model=DenseNet121(include_top=False,weights='imagenet',input_tensor=tf.keras.layers.Input(shape=(200,200,3)))
for layer in pre_trained_model.layers:
    layer.trainable=False

In [41]:
last_layer=pre_trained_model.get_layer('relu')
last_ouput=last_layer.output

In [43]:
x = tf.keras.layers.Flatten()(last_ouput)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(4, activation='softmax')(x)

model_dense=tf.keras.Model(pre_trained_model.input, x)

model_dense.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model_dense.summary()

In [44]:
history=model_dense.fit(train_geneator, epochs=100, callbacks=[checkpoint], validation_data=validation_generator)

In [45]:
epoch=[x for x in range(1,101)]
fig, ax =plt.subplots(1,2,figsize=(12,5))
ax[0].plot(epoch, history.history["accuracy"])
ax[0].plot(epoch, history.history["val_accuracy"])
ax[0].legend(labels=["accuracy","val_accuracy"])
ax[1].plot(epoch, history.history["loss"])
ax[1].plot(epoch, history.history["val_loss"])
ax[1].legend(labels=["loss","val_loss"])
ax[0].set_xlabel("Epoch")
ax[1].set_xlabel("Epoch")
ax[0].set_ylabel("accuracy")
ax[1].set_ylabel("Loss")
fig.suptitle("Model metrics")
plt.show()

Comos vemos en la gráfica de los resultados anteriores, con un modelo pre entrenado obtenemos un accuracy máximo del 93 % en la data de validación. Por otro lado, en el siguiente apartado validaremos el modelo con mi data de test.

In [115]:
print(model_dense.evaluate(train_geneator))
print(model_dense.evaluate(validation_generator))
print(model_dense.evaluate(test_generator))


# conclusiónes

* Logramos desarrollar una red neuronal con un accuracy del 82 % en mis datos de test, cabe aclarar que este modelo presenta overfitting, ya que muestra una diferencia porcentual notoria entre mis datos de Train y mis datos de Test. Sin embargo, logramos obtener un buen resultado en comparación con el desarrollado en el anterior notebook.
* Implementamos con éxito un modelo pre entrenado como lo fue DenseNet, el cual mediante sus ventajas de neuronas y capas profundas realizadas por expertos, logramos mejorar de forma notoria nuestro desempeño dentro de nuestro conjunto de datos.

# recomendaciones 

* Como recomendación general, podemos añadirle más capas a este modelo, donde busquemos reducir de forma considerable el Overfitting mediante Batch normalization o agregando regularizadores.
